In [ ]:
Features and Test Planning

- A clothing and footwear store would like to move management of their items 
  from paper to a new computer the owner bought.
    
  While the owner would like many features, she's content with software that could 
  perform the following upcoming tasks immediately.
    
    . Record the 10 new Nike sneakers that she recently bought. Each is worth $50.00.
    
    . Add 5 more Adidas sweatpants that cost $70.00 each.
    
    . She's expecting a customer to buy 2 of the Nike sneakers.
    
    . She's expecting another customer to buy 1 of the sweatpants.

We can use these requirements to create our first integration test. 
Before we get to writing it, let's flesh out the smaller components a bit to figure out 
what would be our inputs and output, function signatures and other system design elements.

Each item of stock will have a name, price and quantity. 
We'll be able to add new items, add stock to existing items and of course remove stock.

When we instantiate an Inventory object, we'll want the user to provide a limit. 
The limit will have a default value of 100. 
Our first test would be to check the limit when instantiating an object. 
To ensure that we don't go over our limit, we'll need to keep track of the total_items counter. 
When initialized, this should be 0.

We'll need to add 10 Nike sneakers and the 5 Adidas sweatpants to the system. 
We can create an add_new_stock() method that accepts a name, price, and quantity.

We should test that we can add an item to our inventory object. 
We should not be able to add an item with a negative quantity, the method should raise 
an exception. 
We also should not be able to add any more items if we're at the our limit, that should also raise 
an exception.

Customers will be buying these items soon after entry, so we'll need a remove_stock() method 
as well. This function would need the name of the stock and the quantity of items being removed. 
If the quantity being removed is negative or if it makes the total quantity for the stock below 0, 
then the method should raise an exception. 
Additionally, if the name provided is not found in our inventory, the method should raise 
an exception.

In [ ]:
# invent.py
class ItemNotFound(Exception):
    pass


class NoSpaces(Exception):
    pass


class InvalidQuantity(Exception):
    pass


class Invent:
    def __init__(self, limit=100):
        self.limit = limit
        self.total_items = 0
        self.stocks = {}

    def add_new_stock(self, name, price, quantity):
        if quantity <= 0:
            raise InvalidQuantity('Cannot add {} quantity'.format(quantity))

        if self.limit < self.total_items + quantity:
            raise NoSpaces('No Spaces are available')

        diff_quantity = 0

        for n in self.stocks:
            if name == n:  # already existing ?
                diff_quantity = quantity - self.stocks[name]['quantity']

        self.stocks[name] = {
            'price': price,
            'quantity': quantity
        }

        self.total_items += quantity if diff_quantity == 0 else diff_quantity

    def remove_stock(self, name, quantity):
        if quantity <= 0:
            raise InvalidQuantity('Cannot remove 0 quantity')

        if name not in self.stocks:
            raise ItemNotFound(
                "Could not find '{}' in our stocks.".format(name))

        if self.stocks[name]['quantity'] - quantity <= 0:
            raise InvalidQuantity(
                'Cannot remove these {} items. Only {} items are in stock'.format(
                    quantity, self.stocks[name]['quantity']))

        self.stocks[name]['quantity'] -= quantity
        self.total_items -= quantity


In [1]:
#test_invent.py

import pytest
from invent import Invent, InvalidQuantity, NoSpaces, ItemNotFound


@pytest.fixture
def no_stock_invent():
    """Returns empty invent that can store 10 items"""
    return Invent(10)


@pytest.fixture
def ten_stock_invent():
    """Returns an inventory with some test stock items"""
    invent = Invent(20)
    invent.add_new_stock('Puma Test', 80.00, 7)
    invent.add_new_stock('Puma Test', 100.00, 9)
    invent.add_new_stock('Reebok Test', 25.50, 2)
    # print(invent.stocks['Puma Test'])
    # print(invent.total_items)
    return invent


def test_default_invent():
    """Test that default limit is 100"""
    invent = Invent()
    assert invent.limit == 100
    assert invent.total_items == 0


def test_add_new_stock_success(no_stock_invent):
    no_stock_invent.add_new_stock('Test Jacket', 10.00, 5)
    assert no_stock_invent.total_items == 5
    assert no_stock_invent.stocks['Test Jacket']['price'] == 10.00
    assert no_stock_invent.stocks['Test Jacket']['quantity'] == 5


@pytest.mark.parametrize('name,price,quantity,exception', [
    ('Test Jacket', 10.00, 0, InvalidQuantity('Cannot add 0 quantity')),
    ('Test Jacket', 10.00, 20, NoSpaces('No Spaces are available')),
    ('Test Jacket', 10.00, 5, None)
])
def test_add_new_stock_bad(no_stock_invent, name, price, quantity, exception):
    # invent = Invent(10)
    try:
        # invent.add_new_stock(name, price, quantity)
        no_stock_invent.add_new_stock(name, price, quantity)
    except (InvalidQuantity, NoSpaces) as e:
        assert isinstance(e, type(exception))
        assert e.args == exception.args
    else:
        # pytest.fail("Expected error but not found")
        assert no_stock_invent.total_items == quantity
        assert no_stock_invent.stocks[name]['price'] == price
        assert no_stock_invent.stocks[name]['quantity'] == quantity


@pytest.mark.parametrize('name,quantity,exception,new_quantity,new_total', [
    ('Puma Test', 0, InvalidQuantity('Cannot remove 0 quantity'), 0, 0),
    ('Not Here', 5, ItemNotFound("Could not find 'Not Here' in our stocks."), 0, 0),
    ('Puma Test', 25, InvalidQuantity('Cannot remove these 25 items. Only 9 items are in stock'), 0, 0),
    ('Puma Test', 5, None, 4, 6)
])
def test_remove_stock(ten_stock_invent, name, quantity, exception, new_quantity,
                      new_total):
    try:
        ten_stock_invent.remove_stock(name, quantity)
    except (InvalidQuantity, ItemNotFound) as e:
        assert isinstance(e, type(exception))
        assert e.args == exception.args
    else:
        assert ten_stock_invent.stocks[name]['quantity'] == new_quantity
        assert ten_stock_invent.total_items == new_total
